In [105]:
import pandas as pd
import numpy as np

In [106]:
Dat=pd.read_excel('C:/Users\dmckenzie\Downloads\ML Project Room Features Combined.xlsx',sheet_name=4)

In [107]:
Dat=Dat[(Dat["Name_word_count"].str.len()>0)==False]

In [108]:
Dat=Dat[Dat['ProgramArea'].str.contains('\[').isna()]

In [109]:
Dat.to_csv('MLfeatures.csv',index=False)

In [110]:
Dat=pd.read_csv('MLfeatures.csv')

In [112]:
import nltk
def tokennamefilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if x not in ['(e)','(w)','(n)','(s)','east','west','south','north','office','off']]
    return (tokens)

In [113]:
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk import pos_tag

In [114]:
def lemmer(text):
    lemma_function = WordNetLemmatizer()

    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    return [lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(text)]

In [115]:
def tokenlenfilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if len(x)>minlen]
    return (tokens)

In [116]:
Dat['Name']=Dat['Name'].str.lower().str.replace('\d+','').str.replace('\+','').str.replace('[^\w\s]',' ').str.replace('_',' ').\
    str.lstrip().str.rstrip('-').str.rstrip().apply(nltk.word_tokenize).apply(lemmer).apply(tokennamefilter).apply(tokenlenfilter).apply(' '.join).str.lstrip()

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [117]:
Dat=Dat.query("Name!=''").reset_index(drop=True)

In [118]:
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['coffee']))==True),'coffee',Dat['Name'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['officer']))==True),'officer',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['share']))==True),'share',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['principal']))==True),'principal',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['technician']))==True),'technician',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['dean']))==True),'dean',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['nurse']))==True),'nurse',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['work','station']))==True),'work station',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['cust','custodial']))==True),'custodial',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['storage']))==True),'storage',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['teacher']))==True),'teacher',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['admin']))==True),'admin',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['coach']))==True),'coach',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['manager']))==True),'manager',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['hotel']))==True),'hotel',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['faculty']))==True),'faculty',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['staff']))==True),'staff',Dat['Lab'])
Dat['Lab']=np.where((Dat['Name'].str.contains('|'.join(['guest']))==True),'guest',Dat['Lab'])
Dat['Lab'].value_counts().head(18)

private         1613
coffee           115
work station     112
share            106
principal         64
technician        49
manager           48
custodial         42
nurse             41
dean              40
main              40
staff             37
hotel             33
admin             30
officer           30
storage           29
coach             25
guest             25
Name: Lab, dtype: int64

In [119]:
keeplist=['coffee','work station','share','principal','technician','guest','manager','custodial','nurse','main','dean','staff','hotel','officer','admin','storage','coach']             

In [120]:
def get_ngrams(series: pd.Series,n:int):
    ngrams = series.copy().str.split(' ').explode()
    ngrams1 = series.copy().str.split(' ').explode()
    for i in range(1,n):
        foo=ngrams1.groupby(level=0).shift(-i)
        ngrams=ngrams.str.cat(foo,sep=' ')
    ngrams = ngrams.dropna()
    return pd.DataFrame(ngrams)

In [121]:
Dat['Lab'].value_counts().head(17)

private         1613
coffee           115
work station     112
share            106
principal         64
technician        49
manager           48
custodial         42
nurse             41
dean              40
main              40
staff             37
hotel             33
admin             30
officer           30
storage           29
coach             25
Name: Lab, dtype: int64

In [122]:
get_ngrams(Dat['Lab'],1).value_counts().head(20)

Lab       
private       1629
coffee         115
station        112
work           112
share          106
principal       64
technician      49
manager         48
main            44
custodial       42
nurse           41
dean            40
staff           37
dir             34
hotel           33
admin           30
officer         30
storage         29
coach           25
guest           25
dtype: int64

In [123]:
Dat=Dat[Dat['Lab'].isin(keeplist)].reset_index(drop=True)

In [124]:
#len(Dat.describe(include=[np.number]).drop(columns=(['key_0','ModelRefId'])).T)

In [126]:
Dat=Dat.drop(columns=(['key_0','ModelRefId','UpperLimit','perim_area_ratio','area_ratio_bool']))

In [127]:
Dat['Level_Name']=Dat['Level_Name'].str.extract('(\d+)', expand=False)

In [128]:
Dat['Level_Name']=Dat['Level_Name'].astype(float)

In [129]:
Dat.dtypes

UniqueId                        object
Comments                        object
Perimeter                      float64
Area                           float64
Number                          object
Name                            object
UnboundedHeight                float64
LimitOffset                    float64
LowerOffset                    float64
OccupantLoad                   float64
Gross_Net                       object
OccupancyLoadFactor            float64
OccLoadFactor                  float64
OccupancyType                   object
ProgramRoomArea                float64
HearingImpaired                float64
ADARoom                        float64
Capacity                       float64
ISBESF_stu                     float64
ISBESF_stu_Key                 float64
K_12DepartmentSchedule          object
OccupancyLoad                  float64
ConnectorRoom                  float64
TargetProgramArea              float64
ProgramArea                       bool
Name_word_count          

In [130]:
oknums=list(Dat.std().reset_index().rename(columns={0:'std'}).query('std>0')['index'])

In [131]:
allnum=list(Dat.std().reset_index()['index'])

In [132]:
OkOBJ=Dat.drop(columns=allnum).drop(columns=(['UniqueId'])).fillna("BLANKDATA")

In [133]:
for i in list(Dat.columns):
    print(i,': ',Dat[i].astype('str').str.contains('\[').sum())

UniqueId :  0
Comments :  0
Perimeter :  0
Area :  0
Number :  0
Name :  0
UnboundedHeight :  0
LimitOffset :  0
LowerOffset :  0
OccupantLoad :  0
Gross_Net :  0
OccupancyLoadFactor :  0
OccLoadFactor :  0
OccupancyType :  0
ProgramRoomArea :  0
HearingImpaired :  0
ADARoom :  0
Capacity :  0
ISBESF_stu :  0
ISBESF_stu_Key :  0
K_12DepartmentSchedule :  0
OccupancyLoad :  0
ConnectorRoom :  0
TargetProgramArea :  0
ProgramArea :  0
Name_word_count :  0
Name_count :  0
0 :  0
Doors_count :  0
Windows_count :  0
Data Devices_count :  0
CeilingTypeFamilyName_count :  0
Specialty Equipment_count :  0
door_type_not_contain_all :  0
cat_count :  0
door_type_contains_1 :  0
door_type_contains_2 :  0
door_type_contains_3 :  0
Level_Name :  0
room_cats :  0
area_ratio :  0
name_contains_admin :  0
name_contains_gym :  0
Lab :  0


In [134]:
sum(Dat.drop(columns=(['UniqueId'])).std()==0)/len(Dat.drop(columns=(['UniqueId'])).std())

0.22857142857142856

In [151]:
NumDat=Dat[oknums].fillna('-1')

In [152]:
NumDat=NumDat*1

In [153]:
NumDat.apply(pd.to_numeric, errors='ignore').describe().T

,count,mean,std,min,25%,50%,75%,max
Perimeter,866.0,350.422412,3312.014459,17.160583,37.962021,50.017248,73.700670,51579.988231
Area,866.0,194.539495,161.069980,10.894950,78.367653,143.442973,263.539809,982.958978
UnboundedHeight,866.0,38.908491,291.327384,3.000000,8.000000,10.000000,10.000000,4000.000000
LimitOffset,866.0,38.584665,291.358309,-1.200000,8.000000,10.000000,10.000000,4000.000000
LowerOffset,866.0,-0.008083,0.182922,-5.000000,0.000000,0.000000,0.000000,0.000000
OccupantLoad,866.0,0.758661,5.303784,-1.000000,-1.000000,0.000000,1.000000,139.000000
OccupancyLoadFactor,866.0,16.568129,48.967542,-1.000000,-1.000000,0.000000,0.000000,300.000000
OccLoadFactor,866.0,13.729792,45.763513,-1.000000,-1.000000,0.000000,0.000000,300.000000
ProgramRoomArea,866.0,5.670901,33.040791,-1.000000,-1.000000,-1.000000,0.000000,500.000000
OccupancyLoad,866.0,-0.451501,9.288719,-1.000000,-1.000000,-1.000000,-1.000000,157.000000


In [154]:
NumDat=NumDat.apply(pd.to_numeric, errors='ignore')

In [160]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [155]:
y = Dat['Lab']

In [192]:
X =NumDat.drop(columns=['name_contains_admin','name_contains_gym'])

In [196]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 44)

In [237]:
parameters = {
    'clf__C': (6,7,8,9,10,11,12,12,14,15,16),
    'clf__solver': ('liblinear','sag','saga','newton-cg')
}


pipeline = Pipeline([
               ('clf', LogisticRegression(penalty='l2'))
              ])

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
grid_search.fit(X_train, y_train)
from sklearn.metrics import classification_report
y_pred = grid_search.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.4377880184331797
              precision    recall  f1-score   support

       admin       0.17      0.17      0.17         6
       coach       0.00      0.00      0.00         8
      coffee       0.47      0.63      0.54        27
   custodial       0.33      0.33      0.33        12
        dean       0.10      0.09      0.10        11
       guest       0.83      0.62      0.71         8
       hotel       0.18      1.00      0.31         2
        main       0.83      0.83      0.83        12
     manager       0.33      0.25      0.29        16
       nurse       0.00      0.00      0.00        17
     officer       0.43      0.50      0.46         6
   principal       0.07      0.10      0.08        10
       share       0.54      0.70      0.61        20
       staff       0.00      0.00      0.00         7
     storage       0.60      0.25      0.35        12
  technician       0.75      0.75      0.75        12
work station       0.72      0.68      0.70        31

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [214]:
from sklearn.dummy import DummyClassifier

parameters = {
'clf__strategy': ("stratified", "most_frequent", "prior", "uniform", "constant")
}


pipeline = Pipeline([
               ('clf', DummyClassifier(strategy='uniform'))
              ])

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
grid_search.fit(X_train, y_train)
from sklearn.metrics import classification_report
y_pred = grid_search.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.12442396313364056
              precision    recall  f1-score   support

       admin       0.00      0.00      0.00         6
       coach       0.00      0.00      0.00         8
      coffee       0.12      1.00      0.22        27
   custodial       0.00      0.00      0.00        12
        dean       0.00      0.00      0.00        11
       guest       0.00      0.00      0.00         8
       hotel       0.00      0.00      0.00         2
        main       0.00      0.00      0.00        12
     manager       0.00      0.00      0.00        16
       nurse       0.00      0.00      0.00        17
     officer       0.00      0.00      0.00         6
   principal       0.00      0.00      0.00        10
       share       0.00      0.00      0.00        20
       staff       0.00      0.00      0.00         7
     storage       0.00      0.00      0.00        12
  technician       0.00      0.00      0.00        12
work station       0.00      0.00      0.00        3

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [239]:
mod=LogisticRegression(C=7,solver='newton-cg',penalty='l2')
mod.fit(X_train, y_train)
from sklearn.metrics import classification_report
y_pred = mod.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.4377880184331797
              precision    recall  f1-score   support

       admin       0.17      0.17      0.17         6
       coach       0.00      0.00      0.00         8
      coffee       0.47      0.63      0.54        27
   custodial       0.33      0.33      0.33        12
        dean       0.10      0.09      0.10        11
       guest       0.83      0.62      0.71         8
       hotel       0.18      1.00      0.31         2
        main       0.83      0.83      0.83        12
     manager       0.33      0.25      0.29        16
       nurse       0.00      0.00      0.00        17
     officer       0.43      0.50      0.46         6
   principal       0.07      0.10      0.08        10
       share       0.54      0.70      0.61        20
       staff       0.00      0.00      0.00         7
     storage       0.60      0.25      0.35        12
  technician       0.75      0.75      0.75        12
work station       0.72      0.68      0.70        31

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [293]:
pd.DataFrame({'Coef':X.columns,
              'Coef Median':X.apply(np.median).values,
              'Max Coef Score':pd.DataFrame(mod.coef_).apply(max,axis=0),
             'Novice Impact Score':abs(X.apply(np.median).values*pd.DataFrame(mod.coef_).apply(max,axis=0))}).\
    sort_values('Novice Impact Score',ascending=False)
#NumDat.shape

,Coef,Coef Median,Max Coef Score,Novice Impact Score
11,Name_count,15.000000,0.578675,8.680132
1,Area,143.442973,0.042232,6.057945
14,CeilingTypeFamilyName_count,1.000000,3.663612,3.663612
0,Perimeter,50.017248,0.062748,3.138466
2,UnboundedHeight,10.000000,0.305363,3.053630
3,LimitOffset,10.000000,0.292214,2.922136
10,Name_word_count,2.000000,1.381959,2.763917
9,OccupancyLoad,-1.000000,1.603583,1.603583
21,Level_Name,1.000000,0.892008,0.892008
17,cat_count,2.000000,0.104664,0.209327
